# Bowtie Structure

Esta función toma una red como entrada y devuelve una descomposición de conjuntos de nodos según la estructura de lazo de corbata de la red de entrada. La función utiliza el algoritmo descrito en el artículo "Bow-tie decomposition in directed graphs" de R. Yang, L. Zhuhadar y O. Nasraoui.

La función comienza encontrando el componente fuertemente conectado más grande de la red de entrada utilizando la función `max` de la biblioteca `networkx`. A continuación, se selecciona un nodo arbitrario del componente fuertemente conectado más grande y se calculan los nodos alcanzables hacia adelante y hacia atrás desde el componente fuertemente conectado más grande utilizando la función `dfs_tree` de la biblioteca `networkx`.

A continuación, se calculan los componentes de entrada y salida de la estructura de lazo de corbata utilizando los nodos alcanzables hacia adelante y hacia atrás. Los nodos que son alcanzables hacia adelante pero no hacia atrás forman el componente de salida, mientras que los nodos que son alcanzables hacia atrás pero no hacia adelante forman el componente de entrada.

Después de calcular los componentes de entrada y salida, se calculan los componentes de "tendril", "tubo" y "desconectado". Los nodos que son alcanzables hacia adelante y hacia atrás forman un "tubo", los nodos que son alcanzables hacia atrás pero no hacia adelante forman un "tendril" de entrada, los nodos que son alcanzables hacia adelante pero no hacia atrás forman un "tendril" de salida, y los nodos que no son alcanzables ni hacia adelante ni hacia atrás se consideran desconectados.

Finalmente, la función devuelve los conjuntos de nodos para cada componente de la estructura de lazo de corbata: el componente fuertemente conectado más grande, los componentes de entrada y salida, los componentes de "tendril", "tubo" y "desconectado". 

En resumen, esta función se utiliza para descomponer una red en sus componentes de la estructura de lazo de corbata utilizando el algoritmo descrito en el artículo "Bow-tie decomposition in directed graphs". La función devuelve los conjuntos de nodos para cada componente de la estructura de lazo de corbata, lo que puede ser útil para analizar la estructura de la red y comprender su comportamiento.

In [1]:
import networkx as nx

def bowtie_structure(network):
    """ 
    Return node set decomposition according to the bowtie structure of the input network.
    Algorithm from 
    R. Yang, L. Zhuhadar and O. Nasraoui, "Bow-tie decomposition in directed graphs",2011
    """
    
    largest_scc = max(nx.strongly_connected_components(network), key=len)
    
    # Arbitrary node from the largest SCC
    node = next(iter(largest_scc))
    
    # Reachable nodes (forward) from the largest SCC
    dfs = set(nx.dfs_tree(network,node).nodes())
    
    # Reachable nodes (backwards) from the largest SCC
    reversed_network = nx.reverse(network, copy=True)
    dfs_t = set(nx.dfs_tree(reversed_network,node).nodes())
    
    out_component = dfs - largest_scc
    in_component = dfs_t - largest_scc
    
    # Tendrils, tubes and disconnected components
    rest = set(network.nodes()) -  largest_scc - out_component - in_component

    tubes, in_tendrils, out_tendrils, disconnected  = set(), set(), set(), set()

    for v in rest:
        # in_component nodes backwards reachable from v
        irv = in_component & set(nx.dfs_tree(reversed_network, v).nodes())
        # out_component nodes reachable from v
        vro = out_component & set(nx.dfs_tree(network, v).nodes())
        
        if irv and vro:
            tubes.add(v)
        elif irv and not vro:
            in_tendrils.add(v)
        elif not irv and vro:
            out_tendrils.add(v)
        else:
            disconnected.add(v)
            
    return  largest_scc, in_component, out_component, tubes, in_tendrils, out_tendrils, disconnected

### Maven

In [18]:
maven = nx.read_adjlist(r'data/maven-dependencies-net-2020-01-12.bz2', create_using=nx.DiGraph())

In [23]:
largest_scc, in_component, out_component, tubes, in_tendrils, out_tendrils, disconnected = bowtie_structure(maven)

In [24]:
assert len(largest_scc)+\
      len(in_component)+\
      len(out_component)+\
      len(tubes)+\
      len(in_tendrils)+\
      len(out_tendrils)+\
      len(disconnected) == len(maven)

In [25]:
print(len(largest_scc), 
      len(in_component), 
      len(out_component), 
      len(tubes), 
      len(in_tendrils), 
      len(out_tendrils), 
      len(disconnected))

981 2163 70239 3749 10173 24200 15247


### NPM

In [26]:
npm = nx.read_adjlist(r'data/npm-dependencies-net-2020-01-12.bz2', create_using=nx.DiGraph())

In [27]:
largest_scc, in_component, out_component, tubes, in_tendrils, out_tendrils, disconnected = bowtie_structure(npm)

In [28]:
print(len(largest_scc), 
      len(in_component), 
      len(out_component), 
      len(tubes), 
      len(in_tendrils), 
      len(out_tendrils), 
      len(disconnected))

26486 3849 936295 3745 17891 69604 16638


In [29]:
assert len(largest_scc)+\
      len(in_component)+\
      len(out_component)+\
      len(tubes)+\
      len(in_tendrils)+\
      len(out_tendrils)+\
      len(disconnected) == len(npm)
